# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [37]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [38]:
# Create aiport dataframe from previous exercise output
cities_df = pd.read_csv('../output_data/weather_cities_output.csv')
cities_df.dropna(inplace = True) 
cities_df.head()


,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,zhezkazgan,47.8043,67.7144,302.28,24.0,10.0,4.91,KZ,1.627930e+09
1,1,lebu,-37.6167,-73.6500,286.32,72.0,46.0,9.06,CL,1.627929e+09
3,3,upernavik,72.7868,-56.1549,280.94,69.0,10.0,0.58,GL,1.627929e+09
4,4,bossier city,32.5160,-93.7321,301.47,80.0,75.0,0.00,US,1.627929e+09
6,6,inhambane,-23.8650,35.3833,292.81,77.0,0.0,4.40,MZ,1.627930e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [39]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = cities_df[['Lat', 'Lng']]

# Fill NaN values and convert to float
humidity = cities_df['Humidity'].astype(float)

In [42]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=93,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [43]:
hotel_df = cities_df.loc[(cities_df["Max Temp"] <= 283) &
                                                 (cities_df["Max Temp"] >= 270) &
                                                 (cities_df["Wind Speed"] < 10) &
                                                 (cities_df["Cloudiness"] < 10)]
hotel_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
9,9,geraldton,-28.7667,114.6000,280.38,93.0,2.0,4.12,AU,1.627930e+09
103,103,belaya gora,68.5333,146.4167,279.32,61.0,8.0,2.03,RU,1.627930e+09
112,112,colesberg,-30.7200,25.0972,281.92,25.0,0.0,1.13,ZA,1.627930e+09
245,245,yulara,-25.2406,130.9889,277.06,56.0,0.0,3.60,AU,1.627930e+09
259,259,deputatskiy,69.3000,139.9000,276.87,63.0,0.0,1.64,RU,1.627930e+09
289,289,clyde river,70.4692,-68.5914,280.28,76.0,1.0,6.17,CA,1.627930e+09
354,354,broken hill,-31.9500,141.4333,278.99,87.0,0.0,4.63,AU,1.627930e+09
444,444,ola,59.5833,151.2833,280.90,64.0,0.0,4.57,RU,1.627930e+09
519,519,vardo,70.3705,31.1107,282.26,81.0,0.0,2.06,NO,1.627930e+09


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [44]:
#renaming dulpicate column to new Hotel column
hotel_df.rename(columns={ hotel_df.columns[0]: "Hotel" }, inplace = True)

hotel_df

C:\Users\richk\anaconda3\lib\site-packages\pandas\core\frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,Hotel,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
9,9,geraldton,-28.7667,114.6000,280.38,93.0,2.0,4.12,AU,1.627930e+09
103,103,belaya gora,68.5333,146.4167,279.32,61.0,8.0,2.03,RU,1.627930e+09
112,112,colesberg,-30.7200,25.0972,281.92,25.0,0.0,1.13,ZA,1.627930e+09
245,245,yulara,-25.2406,130.9889,277.06,56.0,0.0,3.60,AU,1.627930e+09
259,259,deputatskiy,69.3000,139.9000,276.87,63.0,0.0,1.64,RU,1.627930e+09
289,289,clyde river,70.4692,-68.5914,280.28,76.0,1.0,6.17,CA,1.627930e+09
354,354,broken hill,-31.9500,141.4333,278.99,87.0,0.0,4.63,AU,1.627930e+09
444,444,ola,59.5833,151.2833,280.90,64.0,0.0,4.57,RU,1.627930e+09
519,519,vardo,70.3705,31.1107,282.26,81.0,0.0,2.06,NO,1.627930e+09


In [45]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"
params = {
    "radius": 5000,  
    "type": "lodging",
    "key": g_key,
}

for index, row in hotel_df.iterrows():
    lat = row.Lat
    lng = row.Lng
    params["location"] = f'{lat}, {lng}'
    response = requests.get(base_url,params).json()
    try:
        hotel_df.loc[index,'Hotel'] = response['results'][0]['name']
        
    #also for cities not found in the open weather to be skipped and show progress
    except (KeyError, IndexError):
           print("Hotel not found. Skipping...")
   
hotel_df

C:\Users\richk\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


Hotel not found. Skipping...
Hotel not found. Skipping...


,Hotel,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
9,Broadwater Mariner Resort,geraldton,-28.7667,114.6000,280.38,93.0,2.0,4.12,AU,1.627930e+09
103,103,belaya gora,68.5333,146.4167,279.32,61.0,8.0,2.03,RU,1.627930e+09
112,Bhelekazi Guesthouse,colesberg,-30.7200,25.0972,281.92,25.0,0.0,1.13,ZA,1.627930e+09
245,Desert Gardens Hotel - Ayers Rock Resort,yulara,-25.2406,130.9889,277.06,56.0,0.0,3.60,AU,1.627930e+09
259,Baza Otdykha,deputatskiy,69.3000,139.9000,276.87,63.0,0.0,1.64,RU,1.627930e+09
289,Qamaq Hotel,clyde river,70.4692,-68.5914,280.28,76.0,1.0,6.17,CA,1.627930e+09
354,Royal Exchange Hotel,broken hill,-31.9500,141.4333,278.99,87.0,0.0,4.63,AU,1.627930e+09
444,444,ola,59.5833,151.2833,280.90,64.0,0.0,4.57,RU,1.627930e+09
519,Vardø Hotel AS,vardo,70.3705,31.1107,282.26,81.0,0.0,2.06,NO,1.627930e+09


In [46]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap  ##had to edit hotel name to hotel to fit my formatting
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [47]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)



# Display figure
fig = gmaps.figure()
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))